In [14]:
import fitz
verdict_doc = "Landmark_hearing.pdf"

with fitz.open(verdict_doc) as doc:
  text = ''
  for page in doc:
     c_text = page.get_text()
     c_text = c_text.replace('\n', ' ')
     text += c_text

In [15]:
len(text)

38158

In [16]:
from langchain_cerebras import ChatCerebras
from langchain import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 5000, chunk_overlap = 20)
chunks = text_splitter.create_documents([text])

In [18]:
chunks

[Document(metadata={}, page_content="2024 INSC 739 Reportable IN\xa0THE\xa0SUPREME\xa0COURT\xa0OF\xa0INDIA CRIMINAL\xa0APPELLATE\xa0JURISDICTION CRIMINAL\xa0APPEAL\xa0NO.4011\xa0OF\xa02024 (Arising\xa0out\xa0of\xa0Special\xa0Leave\xa0Petition\xa0(Crl.)\xa0No.\xa03986\xa0of\xa02024) V.\xa0Senthil\xa0Balaji \xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0…\xa0Appellant versus The\xa0Deputy\xa0Director,\xa0Directorate\xa0of\xa0 Enforcement \xa0\xa0\xa0\xa0\xa0\xa0...\xa0Respondent J\xa0\xa0U\xa0\xa0D\xa0\xa0G\xa0\xa0M\xa0\xa0E\xa0\xa0N\xa0\xa0T ABHAY\xa0S.\xa0OKA,\xa0J. FACTUAL\xa0ASPECTS 1. Leave\xa0granted.\xa0 2. This\xa0appeal\xa0takes\xa0exception\xa0to\xa0the\xa0judgment\xa0and\xa0order dated\xa028th\xa0February\xa02024\xa0passed\xa0by\xa0a\xa0learned\xa0Single\xa0Judge\xa0of the \xa0 High \xa0 Court \xa0 of \xa0 Judicature \xa0 at \xa0 Madras \xa0 by \xa0 which \xa0 a \xa0 bail application\xa0preferred\xa0by\xa0the\xa0appellant\xa0under\xa0Section\xa0439\xa0of the\xa0Code\xa0of\xa0Crimina

In [19]:
import os
from credential import API_KEY
if "CEREBRAS_API_KEY" not in os.environ:
    os.environ["CEREBRAS_API_KEY"] = API_KEY

In [20]:
llm = ChatCerebras(
    model="llama3.1-70b",
    temperature=0
)

In [35]:
from langchain_cerebras import ChatCerebras
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Initialize the model
llm = ChatCerebras(
    model="llama3.1-70b",
    # other params...
)

# Set up the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that summarises the given content with whatever fully available information, start with Summary.",
        ),
        ("human", "{input}"),
    ]
)

# Define the chain
chain = prompt | llm

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 10000, chunk_overlap = 20)
chunks = text_splitter.create_documents([text])

# Split the input text into chunks
# input_text = text # Example text
# chunks = text_splitter.split_text(input_text)

# Process each chunk with the chain
# results = " "
for chunk in chunks:
    response = chain.invoke(
        {
            "input": chunk,
        }
    )
     
print(response.content)


Summary:

The Supreme Court of India allowed the appeal of an accused person and granted bail in a Prevention of Money Laundering Act (PMLA) case, citing the principle of "bail is the rule, and jail is the exception." The court held that the stringent provisions for the grant of bail under the PMLA cannot be used to incarcerate the accused without trial for an unreasonably long time. The court also stated that the power to grant bail lies with the Constitutional Courts, which can exercise their jurisdiction under Article 32 or Article 226 of the Constitution to grant bail on the grounds of violation of Part III of the Constitution.

Key Points:

1. The court held that the delay in the conclusion of the trial and the higher threshold for the grant of bail cannot go together.
2. The court stated that the power to grant bail lies with the Constitutional Courts, which can exercise their jurisdiction under Article 32 or Article 226 of the Constitution.
3. The court held that the stringent p

In [33]:
type(response.content)

str

In [36]:
with open("summary_with_map_reduce.txt", 'w') as file:
    file.write(response.content)
file.close()